# Final Project

Importing necessary packages

In [2]:
import xpress as xp
import pandas as pd
import numpy as np

#### Defining a problem

In [3]:
prob = xp.problem(name="BikeStationsSimple")

/var/folders/qr/mt35g7rx5s78fytrjmk0s9rh0000gn/T/ipykernel_69504/3827034396.py:1: LicenseWarning: Using the license file found in your Xpress installation. If you want to use this license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')
  prob = xp.problem(name="BikeStationsSimple")


#### Sets

In [4]:
zones_df = pd.read_csv("zone_predictions_filtered_poi_gt_0.csv")

In [5]:
# --- index sets ---
J = zones_df['zone_id']  # candidate stations
I = zones_df['zone_id']  # demand zones
T = ['morning', 'afternoon', 'evening', 'night']

In [6]:
commercial_zone_ids = [
    387, 414, 415, 416, 443, 444, 452, 453,
    481, 482, 483, 510, 511, 512, 538, 539, 540,
    564, 565, 566, 589, 590, 611, 612, 613
]

In [7]:
import pandas as pd

group_files = [f"group_{k}_new.csv" for k in range(1, 11)]

zone_groups = []  # this will become a list of lists: [group1, group2, ... group5]
zone_groups_all = []
for f in group_files:
    df = pd.read_csv(f)
    group = list((set(df["zone_id"].tolist()).intersection(I)))
    zone_groups.append(group)
    zone_groups_all.extend(group)

##### Accessibility Parameters

In [8]:
def accessibility_exponential(zones, beta=0.8):
    """
    zones: DataFrame with columns ['zone_id', 'latitude', 'longitude']
    beta:  decay parameter for exp(-beta * distance_km)

    Returns:
        dist_dict  : {(zone_id_i, zone_id_j): distance_km}
        access_dict: {(zone_id_i, zone_id_j): exp(-beta * distance_km)}
    """
    
    # Extract arrays
    zone_ids = zones['zone_id'].values
    lat = np.radians(zones['latitude'].values)
    lon = np.radians(zones['longitude'].values)
    
    # Broadcast
    lat_i = lat[:, None]
    lat_j = lat[None, :]
    lon_i = lon[:, None]
    lon_j = lon[None, :]
    
    # Haversine
    R = 6371.0
    dlat = lat_j - lat_i
    dlon = lon_j - lon_i

    a = (np.sin(dlat/2)**2 +
         np.cos(lat_i) * np.cos(lat_j) * np.sin(dlon/2)**2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    dist_km = R * c
    access = np.exp(-beta * dist_km)

    # Build dictionaries using zone_ids
    dist_dict = {
        (zone_ids[i], zone_ids[j]): float(dist_km[i, j])
        for i in range(len(zone_ids))
        for j in range(len(zone_ids))
    }

    access_dict = {
        (zone_ids[i], zone_ids[j]): float(access[i, j])
        for i in range(len(zone_ids))
        for j in range(len(zone_ids))
    }

    return dist_dict, access_dict

#### Demand

In [9]:
import pandas as pd
import math

time_periods = ['morning', 'afternoon', 'evening', 'night']
types = ['arrivals', 'departures']

demand = {}

for tp in time_periods:
    # Load arrivals + departures CSVs
    f_arr = f"zone_{tp}_arrivals_predictions_daily.csv"
    f_dep = f"zone_{tp}_departures_predictions_daily.csv"

    df_arr = pd.read_csv(f_arr)
    df_dep = pd.read_csv(f_dep)

    # Extract only needed columns
    df_arr = df_arr[['zone_id', f'predicted_{tp}_arrivals_daily']]
    df_dep = df_dep[['zone_id', f'predicted_{tp}_departures_daily']]

    # Merge on zone_id
    merged = df_arr.merge(df_dep, on='zone_id')

    # Compute total predicted flow
    merged['total_predicted'] = (
        merged[f'predicted_{tp}_arrivals_daily'] +
        merged[f'predicted_{tp}_departures_daily']
    )

    # Store in dictionary
    for _, row in merged.iterrows():
        key = (int(row.zone_id), tp)
        value = math.ceil(float(row.total_predicted)/184*1)
        demand[key] = value

#### Service Rate

In [10]:
import pandas as pd
import math

# --- read & clean ---
df = pd.read_csv("Service_rate_time_period.csv")
df.columns = df.columns.str.strip()

df = df.rename(columns={
    "Unnamed: 4": "morning",
    "Unnamed: 8": "afternoon",
    "Unnamed: 12": "evening",
    "Unnamed: 16": "night"
})

# --- build dictionary ---
theta = {}
time_periods = ["morning", "afternoon", "evening", "night"]

for _, row in df.iterrows():
    for time_period in time_periods:
        key = (int(row.zone_id), time_period)
        value = math.ceil(float(row[time_period]))          # or ceil/round if needed
        theta[key] = value

In [11]:
# Two scenarios: weekday vs weekend
S = ["weekday", "weekend"]

# Probabilities (5 weekdays, 2 weekend days per week)
p = {
    "weekday": 5/7,
    "weekend": 2/7,
}

# Time-period specific multipliers for each scenario
# T = ['morning', 'afternoon', 'evening', 'night']
scales = {
    "weekday": {
        "morning":   1.3,   # higher in weekday morning
        "afternoon": 0.9,
        "evening":   1.3,   # higher in weekday evening
        "night":     0.9,
    },
    "weekend": {
        "morning":   0.8,
        "afternoon": 1.3,   # higher in weekend afternoon
        "evening":   1.0,
        "night":     1.3,   # higher in weekend night
    },
}

# Build scenario-specific demand:
# demand_per_zone_s[(i, t, s)] = demand in zone i, time t, scenario s
demand_per_zone_s = {}

for i in I:
    for t in T:
        base = demand[(i, t)]  # existing deterministic demand
        for s in S:
            demand_per_zone_s[(i, t, s)] = scales[s][t] * base

#### Decision Variables

In [12]:
# x_j: open station j (0/1)
x = {j: xp.var(name=f"x_{j}", vartype=xp.binary) for j in J}

# c_j: docks at station j (integer)
c = {j: xp.var(name=f"c_{j}", vartype=xp.integer, lb=0) for j in J}

# (NEW) y_ijts: trips from zone i served by station j, time t, scenario s 
y = {
    (i, j, t, s): xp.var(name=f"y_{i}_{j}_{t}_{s}", vartype=xp.integer, lb=0)
    for i in I for j in J for t in T for s in S
}

# z_i: commercial zone i reach target (0/1)
z = {i: xp.var(name=f"z_{i}", vartype=xp.binary)
     for i in J}

prob.addVariable(list(x.values()) +
                 list(c.values()) +
                 list(y.values()) +
                 list(z.values()))

/var/folders/qr/mt35g7rx5s78fytrjmk0s9rh0000gn/T/ipykernel_69504/1532531873.py:2: DeprecationWarning: Deprecated in Xpress 9.5: create a linked variable by calling problem.addVariable()
  x = {j: xp.var(name=f"x_{j}", vartype=xp.binary) for j in J}
/var/folders/qr/mt35g7rx5s78fytrjmk0s9rh0000gn/T/ipykernel_69504/1532531873.py:5: DeprecationWarning: Deprecated in Xpress 9.5: create a linked variable by calling problem.addVariable()
  c = {j: xp.var(name=f"c_{j}", vartype=xp.integer, lb=0) for j in J}
/var/folders/qr/mt35g7rx5s78fytrjmk0s9rh0000gn/T/ipykernel_69504/1532531873.py:9: DeprecationWarning: Deprecated in Xpress 9.5: create a linked variable by calling problem.addVariable()
  (i, j, t, s): xp.var(name=f"y_{i}_{j}_{t}_{s}", vartype=xp.integer, lb=0)
/var/folders/qr/mt35g7rx5s78fytrjmk0s9rh0000gn/T/ipykernel_69504/1532531873.py:14: DeprecationWarning: Deprecated in Xpress 9.5: create a linked variable by calling problem.addVariable()
  z = {i: xp.var(name=f"z_{i}", vartype=xp.bin

#### Parameters

In [13]:
# ==============================
#  MODEL PARAMETERS
# ==============================

# --- Financial parameters ---
max_budget = 3_000_000        # total budget [£]
fixed_cost = 6_000            # station setup cost [£ per station]
dock_cost = 4_000             # dock installation cost [£ per dock]

# --- Capacity parameters ---
dock_max = 15                 # maximum docks per station
dock_min = 3                  # minimum docks if opened

# --- Service Rates ---
service_rate = theta          # trips per dock per day

# --- Stations parameters ---
station_min = 0               # minimum number of stations
station_max = 120             # maximum number of stations

# --- Demand parameters ---
demand_per_zone = demand      # demand per zone

# --- Accessibiility parameters ---
distance,accessibility = accessibility_exponential(zones_df)

# --- Spatial spacing (exclude commercial zones) ---
min_space = 0.7   # minimal allowed distance between stations [km]

pairs = []

for zi in I:
    for zj in J:
        if zi >= zj:    # avoid duplicates + self-pairs
            continue

        # skip commercial zones
        if zi in commercial_zone_ids or zj in commercial_zone_ids:
            continue

        # check spacing rule using zone_id pairs
        if distance[(zi, zj)] < min_space:
            pairs.append([zi, zj])
            
# --- Commercial parameters ---
commercial_target = 0.9                 # minimal demand target for commercial financial support
subsidy_per_zone = 2000

# --- Zone Groups ---
station_min_zone_group = 4                 # minimal number of station for each zone group

#### Objective Function

In [14]:
prob.setObjective(
    xp.Sum(
        p[s] * y[i, j, t, s]
        for s in S for i in I for j in J for t in T
    ),
    sense=xp.maximize
)

#### Constraints

In [15]:
# ==========================
#        CONSTRAINTS
# ==========================

# (1) Budget
prob.addConstraint(
    xp.Sum(fixed_cost * x[j] + dock_cost * c[j] for j in J)
    - xp.Sum(subsidy_per_zone * z[i] for i in commercial_zone_ids)
    + xp.Sum(0.25 * (fixed_cost * x[j]) for j in zone_groups_all)
    <= max_budget
)

# (2) Dock linking (upper & minimum when open)
prob.addConstraint([c[j] <= dock_max * x[j] for j in J])
prob.addConstraint([c[j] >= dock_min * x[j] for j in J])

# (3) Demand conservation (per zone, per time period, per scenario)
prob.addConstraint([
    xp.Sum(y[i, j, t, s] for j in J) <= demand_per_zone_s[(i, t, s)]
    for i in I for t in T for s in S
])

# (4) Station service capacity (per station, per time period, per scenario)
prob.addConstraint([
    xp.Sum(y[i, j, t, s] for i in I) <= service_rate[j, t] * c[j]
    for j in J for t in T for s in S
])

# (5) Spacing within min_space km (first-stage only)
prob.addConstraint([x[j] + x[k] <= 1 for (j, k) in pairs])

# (6) Accessibility constraint (per i, j, t, scenario)
prob.addConstraint([
    y[i, j, t, s] <= accessibility[i, j] * demand_per_zone_s[(i, t, s)]
    for i in I for j in J for t in T for s in S
])

# (7) Number of stations (first-stage)
prob.addConstraint(xp.Sum(x[j] for j in J) >= station_min)
prob.addConstraint(xp.Sum(x[j] for j in J) <= station_max)

# (8) Commercial district constraints (scenario-wise)
prob.addConstraint([
    xp.Sum(y[i, j, t, s] for j in J) >= commercial_target * demand_per_zone_s[(i, t, s)] * z[i]
    for i in commercial_zone_ids for t in T for s in S
])

# (9) Zone group station constraints (first-stage)
prob.addConstraint([
    xp.Sum(x[j] for j in zone_groups[g]) >= station_min_zone_group
    for g in range(len(zone_groups))
])

#### Solve

In [16]:
# write & solve
prob.write("problem", "lp")
xp.setOutputEnabled(True)

# --- SOLVER CONTROLS AND LIMITS ---

# Time limit (in seconds)
prob.setControl('MAXTIME', 600)          # stop after 10 minutes

# Node limit (branch-and-bound iterations)
prob.setControl('MAXNODE', 15000)         # stop after 15000 explored nodes

# MIP gap tolerance (relative)
prob.setControl('MIPRELSTOP', 0.02)      # stop when within 2% of optimality

# Optional: control verbosity (0 = silent, 1 = summary, 2 = full)
prob.setControl('OUTPUTLOG', 1)

# --- SOLVE AND REPORT ---
print("🚀 Starting solver with runtime and gap limits ...")
prob.solve()

print("\n=== SOLVER SUMMARY ===")
print("Status:", prob.getProbStatusString())

# MIP gap (if available)
try:
    print("Relative MIP gap:", prob.getAttrib('miprelgap'))
except:
    print("No MIP gap info available (non-MIP model).")

print("Objective value:", prob.getObjVal())
print("Solver time (sec):", prob.getAttrib('time'))
print("Nodes explored:", prob.getAttrib('nodes'))

🚀 Starting solver with runtime and gap limits ...
FICO Xpress v9.7.0, Hyper, solve started 0:10:14, Nov 26, 2025
Heap usage: 696MB (peak 696MB, 183MB system)
Maximizing MILP BikeStationsSimple using up to 14 threads and up to 24GB memory, with these control settings:
MAXNODE = 15000
MAXTIME = 600
OUTPUTLOG = 1
MIPRELSTOP = .02
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
   1181440 rows      1174661 cols      3605400 elements   1174661 entities
Presolved problem has:
      6679 rows        47856 cols       100814 elements     47841 entities
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 1097MB (peak 1892MB, 183MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 7.20e-01,  7.50e+03] / [ 6.25e-02,  1.88e+00]
  RHS and bounds [min,max] : [ 7.00e-08,  3.00e+06] / [ 1.00e+00,  1.88e+02]
  Objective      [min,max] : [ 2.86e-01,  7.14e-01] / [ 2.86e-01,  2.00e+00]
Auto

/var/folders/qr/mt35g7rx5s78fytrjmk0s9rh0000gn/T/ipykernel_69504/3127909656.py:24: DeprecationWarning: Deprecated in Xpress 9.5: use problem.attributes.solvestatus and problem.attributes.solstatus instead
  print("Status:", prob.getProbStatusString())
/var/folders/qr/mt35g7rx5s78fytrjmk0s9rh0000gn/T/ipykernel_69504/3127909656.py:32: DeprecationWarning: Deprecated in Xpress 9.5: use problem.attributes.objval instead
  print("Objective value:", prob.getObjVal())


In [19]:
import numpy as np      # numerical helpers
import pandas as pd     # tables / DataFrames

# ANSI colours for pretty printing
BLUE = "\033[94m"
BOLD = "\033[1m"
END  = "\033[0m"

# ==============================================================
# 0) INDEX SETS AND MAPS
# ==============================================================

I_list = list(I)        # zones
J_list = list(J)        # candidate stations
T_list = list(T)        # time periods
S_list = list(S)        # scenarios

i_index = {i: idx for idx, i in enumerate(I_list)}   # zone i → row index
j_index = {j: idx for idx, j in enumerate(J_list)}   # station j → col index
t_index = {t: idx for idx, t in enumerate(T_list)}   # time t → depth index
s_index = {s: idx for idx, s in enumerate(S_list)}   # scenario s → scenario index

# ==============================================================
# 1) x_j AND c_j (TIME-INDEPENDENT), THEN EXPAND ACROSS T
# ==============================================================

# --- x_j: station open decision (0/1) --------------------------
x_keys = list(x.keys())
x_vals = prob.getSolution([x[j] for j in x_keys])
open_base = pd.Series(x_vals, index=x_keys, name="open")   # index = zone_id j

# --- c_j: number of docks at station j ------------------------
c_keys = list(c.keys())
c_vals = prob.getSolution([c[j] for j in c_keys])
docks_base = pd.Series(c_vals, index=c_keys, name="docks") # index = zone_id j

# MultiIndex for (zone_id, time_period)
idx_jt = pd.MultiIndex.from_product(
    [open_base.index, T_list],
    names=["zone_id", "time_period"]
)

# expand x over time
open_df = pd.Series(
    np.repeat(open_base.values, len(T_list)),
    index=idx_jt,
    name="open"
).to_frame()

# expand c over time
docks_df = pd.Series(
    np.repeat(docks_base.values, len(T_list)),
    index=idx_jt,
    name="docks"
).to_frame()

# ==============================================================
# 2) y_ijts → 4D ARRAY, THEN EXPECTED FLOWS OVER SCENARIOS
# ==============================================================

y_keys = list(y.keys())                        # each key is (i, j, t, s)
y_vals = prob.getSolution([y[k] for k in y_keys])

# Y[i, j, t, s] container
Y = np.zeros(
    (len(I_list), len(J_list), len(T_list), len(S_list)),
    dtype=float
)

for val, (i, j, t, s) in zip(y_vals, y_keys):
    Y[i_index[i], j_index[j], t_index[t], s_index[s]] = val

# scenario probability vector aligned with S_list
p_vec = np.array([p[s] for s in S_list], dtype=float)   # length = |S|

# expected flows: Y_exp[i, j, t] = Σ_s p_s * Y[i, j, t, s]
Y_exp = (Y * p_vec[None, None, None, :]).sum(axis=3)

# trips from stations: sum over i → shape (|J|, |T|)
trips_from_stations = Y_exp.sum(axis=0)

# trips serving zones: sum over j → shape (|I|, |T|)
trips_serving_zones = Y_exp.sum(axis=1)

# flatten into MultiIndex for (zone_id, time_period)
idx_jt = pd.MultiIndex.from_product(
    [J_list, T_list],
    names=["zone_id", "time_period"]
)
trips_from_stations_df = (
    pd.Series(
        trips_from_stations.reshape(-1),
        index=idx_jt,
        name="trips_from_stations"
    )
    .to_frame()
)

idx_it = pd.MultiIndex.from_product(
    [I_list, T_list],
    names=["zone_id", "time_period"]
)
trips_serving_zones_df = (
    pd.Series(
        trips_serving_zones.reshape(-1),
        index=idx_it,
        name="trips_serving_zones"
    )
    .to_frame()
)

# ==============================================================
# 3) EXPECTED DEMAND PER (ZONE, TIME PERIOD)
# ==============================================================

# demand_per_zone_s: dict with keys (i, t, s) and value d[i,t,s]
# build expected demand: D_exp[i,t] = Σ_s p_s * d[i,t,s]
demand_exp = {}
for i in I_list:
    for t in T_list:
        demand_exp[(i, t)] = sum(
            p[s] * demand_per_zone_s[(i, t, s)]
            for s in S_list
        )

demand_df = (
    pd.Series(demand_exp, name="Demand")
      .rename_axis(["zone_id", "time_period"])
      .to_frame()
)

# ==============================================================
# 4) BUILD TIME-DEPENDENT SOLUTION DATAFRAME sol_t (EXPECTED)
# ==============================================================

sol_t = (
    demand_df
    .join(trips_serving_zones_df, how="left")
    .join(trips_from_stations_df, how="left")
    .join(open_df, how="left")
    .join(docks_df, how="left")
)

# service rate per zone and time: fraction of demand served
sol_t["Rate"] = sol_t["trips_serving_zones"] / sol_t["Demand"]

# add geometry (longitude / latitude) – no time dependence in zones_df
sol_t = sol_t.reset_index().merge(
    zones_df[["zone_id", "longitude", "latitude"]],
    on="zone_id",
    how="left"
)

# optional: save to CSV for later plotting / analysis
sol_t.to_csv("solution_by_zone_and_time_expected.csv", index=False)

# ==============================================================
# 5) AGGREGATE BACK TO ZONE-LEVEL (EXPECTED) → sol
# ==============================================================

sol = (
    sol_t.groupby("zone_id")
         .agg({
             "open":               "max",   # station either open or not
             "docks":              "max",   # assume docks constant across time
             "trips_from_stations":"sum",   # sum over time
             "trips_serving_zones":"sum",   # sum over time
             "Demand":             "sum",   # total demand over time
         })
         .reset_index()
)

# overall expected service rate per zone
sol["Rate"] = sol["trips_serving_zones"] / sol["Demand"]

# open stations subset
sol_open = sol[sol["open"] == 1]

# masks for zone types
commercial_mask = sol["zone_id"].isin(commercial_zone_ids)
peripheral_mask = sol["zone_id"].isin(zone_groups_all)   # list of peripheral zone_ids

# --- station counts --------------------------------------------
n_stations_all        = len(sol_open)
n_stations_commercial = sol_open["zone_id"].isin(commercial_zone_ids).sum()
n_stations_peripheral = sol_open["zone_id"].isin(zone_groups_all).sum()

# --- dock statistics -------------------------------------------
min_docks    = sol_open["docks"].min()
avg_docks    = sol_open["docks"].mean()
median_docks = sol_open["docks"].median()
max_docks    = sol_open["docks"].max()
std_docks    = sol_open["docks"].std()
total_docks  = sol_open["docks"].sum()

# --- demand-weighted coverage (ALL / COMMERCIAL / PERIPHERAL) --
avg_rate_all = (
    sol["trips_serving_zones"].sum() / sol["Demand"].sum()
) * 100

avg_rate_commercial = (
    sol.loc[commercial_mask, "trips_serving_zones"].sum() /
    sol.loc[commercial_mask, "Demand"].sum()
) * 100

avg_rate_peripheral = (
    sol.loc[peripheral_mask, "trips_serving_zones"].sum() /
    sol.loc[peripheral_mask, "Demand"].sum()
) * 100

# zones with some demand served (0 < Rate ≤ 1)
n_zones_covered = len(sol[(sol["Rate"] > 0) & (sol["Rate"] <= 1)])

# objective value from solver (total trips in model)
obj_trips = int(prob.attributes.objval)

# ==============================================================
# 6) SPATIAL ACCESSIBILITY: NEAREST-NEIGHBOUR DISTANCES
# ==============================================================

# open stations with coordinates
stations = sol_open.merge(
    zones_df[["zone_id", "latitude", "longitude"]],
    on="zone_id",
    how="left"
)

def avg_nearest_neighbor_distance(df):
    """
    Average distance (km) from each station to its nearest other station.
    Uses haversine distance on latitude/longitude.
    """
    n = len(df)
    if n < 2:
        return np.nan

    # convert to radians
    lat = np.radians(df["latitude"].values)
    lon = np.radians(df["longitude"].values)

    # pairwise differences via broadcasting
    lat_i = lat[:, None]
    lat_j = lat[None, :]
    lon_i = lon[:, None]
    lon_j = lon[None, :]

    # haversine formula
    R = 6371.0  # Earth radius in km
    dlat = lat_j - lat_i
    dlon = lon_j - lon_i
    a = np.sin(dlat / 2) ** 2 + np.cos(lat_i) * np.cos(lat_j) * np.sin(dlon / 2) ** 2
    dist = 2 * R * np.arcsin(np.sqrt(a))

    # ignore self-distances
    np.fill_diagonal(dist, np.inf)

    # nearest neighbour for each station, then mean
    nearest = dist.min(axis=1)
    return float(nearest.mean())

# masks within station set
stations_commercial = stations[stations["zone_id"].isin(commercial_zone_ids)]
stations_peripheral = stations[stations["zone_id"].isin(zone_groups_all)]

# average nearest-neighbour distances (km)
avg_nn_all        = avg_nearest_neighbor_distance(stations)
avg_nn_commercial = avg_nearest_neighbor_distance(stations_commercial)
avg_nn_peripheral = avg_nearest_neighbor_distance(stations_peripheral)

def km_to_times(d):
    """
    Convert km to walking and cycling times (minutes).
    Assumes 5 km/h walking, 15 km/h cycling.
    """
    if np.isnan(d):
        return np.nan, np.nan
    walk_min = d * 12   # 60 / 5
    bike_min = d * 4    # 60 / 15
    return walk_min, bike_min

walk_all,  bike_all  = km_to_times(avg_nn_all)
walk_comm, bike_comm = km_to_times(avg_nn_commercial)
walk_peri, bike_peri = km_to_times(avg_nn_peripheral)

# ==============================================================
# 7) COVERAGE BY TIME PERIOD AND ZONE TYPE (FROM sol_t)
# ==============================================================

# masks on sol_t (time-dependent)
mask_comm_t   = sol_t["zone_id"].isin(commercial_zone_ids)
mask_periph_t = sol_t["zone_id"].isin(zone_groups_all)

def coverage_by_time(df):
    """
    Demand-weighted coverage per time_period:
    Σ trips_serving_zones / Σ Demand within each time_period.
    """
    return (
        df.groupby("time_period")
          .apply(lambda d: d["trips_serving_zones"].sum() / d["Demand"].sum())
    )

# coverage series by time (fractions 0–1)
cov_all    = coverage_by_time(sol_t)
cov_comm   = coverage_by_time(sol_t[mask_comm_t])
cov_periph = coverage_by_time(sol_t[mask_periph_t])

# combine into one coverage table (%) and order by T_list
coverage_table = pd.DataFrame({
    "All zones":    cov_all,
    "Commercial":   cov_comm,
    "Peripheral":   cov_periph,
}) * 100
coverage_table = coverage_table.reindex(T_list)

# overall coverage (all periods) per zone type (%)
overall_all = sol_t["trips_serving_zones"].sum() / sol_t["Demand"].sum() * 100
overall_comm = (
    sol_t[mask_comm_t]["trips_serving_zones"].sum() /
    sol_t[mask_comm_t]["Demand"].sum() * 100
)
overall_periph = (
    sol_t[mask_periph_t]["trips_serving_zones"].sum() /
    sol_t[mask_periph_t]["Demand"].sum() * 100
)

# ==============================================================
# 8) PRINT ALL KEY RESULTS (ONE CELL OUTPUT)
# ==============================================================

# --- station counts & docks ------------------------------------
print(f"Number of stations opened (all zones): {BLUE}{BOLD}{n_stations_all}{END}")
print(f"Number of stations in commercial zones: {BLUE}{BOLD}{n_stations_commercial}{END}")
print(f"Number of stations in peripheral zones: {BLUE}{BOLD}{n_stations_peripheral}{END}")

print(f"\nMin number of docks per open station: {BLUE}{BOLD}{min_docks:.0f}{END} docks")
print(f"Avg number of docks per open station: {BLUE}{BOLD}{avg_docks:.1f}{END} docks")
print(f"Median number of docks per open station: {BLUE}{BOLD}{median_docks:.1f}{END} docks")
print(f"Max number of docks per open station: {BLUE}{BOLD}{max_docks:.0f}{END} docks")
print(f"Std of docks per open station: {BLUE}{BOLD}{std_docks:.1f}{END} docks")
print(f"Total number of docks: {BLUE}{BOLD}{total_docks:.0f}{END} docks")

# --- demand coverage (zone-level) ------------------------------
print(f"\nAverage demand satisfied (all zones): "
      f"{BLUE}{BOLD}{avg_rate_all:.1f}%{END}")
print(f"Average demand satisfied in commercial zones: "
      f"{BLUE}{BOLD}{avg_rate_commercial:.1f}%{END}")
print(f"Average demand satisfied in peripheral zones: "
      f"{BLUE}{BOLD}{avg_rate_peripheral:.1f}%{END}")

print(f"\nNumber of zones covered: {BLUE}{BOLD}{n_zones_covered}{END}")
print(f"Demand coverage rate (all zones): "
      f"{BLUE}{BOLD}{avg_rate_all:.1f}%{END}")
print(f"Number of trips (objective function value): "
      f"{BLUE}{BOLD}{obj_trips}{END} trips")

# --- spatial accessibility -------------------------------------
print(f"\nAvg distance to nearest open station (ALL): "
      f"{BLUE}{BOLD}{avg_nn_all:.2f}{END} km "
      f"(≈{walk_all:.0f} min walking, ≈{bike_all:.0f} min by bike)")
print(f"Avg distance to nearest open station (COMMERCIAL): "
      f"{BLUE}{BOLD}{avg_nn_commercial:.2f}{END} km "
      f"(≈{walk_comm:.0f} min walking, ≈{bike_comm:.0f} min by bike)")
print(f"Avg distance to nearest open station (PERIPHERAL): "
      f"{BLUE}{BOLD}{avg_nn_peripheral:.2f}{END} km "
      f"(≈{walk_peri:.0f} min walking, ≈{bike_peri:.0f} min by bike)")

# --- time-period coverage table --------------------------------
print("\nDemand coverage by time period and zone type (%):")
print(coverage_table.round(1))

print("\nOverall demand coverage (all periods):")
print(f"  All zones:      {overall_all:5.1f}%")
print(f"  Commercial:     {overall_comm:5.1f}%")
print(f"  Peripheral:     {overall_periph:5.1f}%")

Number of stations opened (all zones): 97
Number of stations in commercial zones: 9
Number of stations in peripheral zones: 69

Min number of docks per open station: 3 docks
Avg number of docks per open station: 5.9 docks
Median number of docks per open station: 4.0 docks
Max number of docks per open station: 15 docks
Std of docks per open station: 3.9 docks
Total number of docks: 575 docks

Average demand satisfied (all zones): 73.8%
Average demand satisfied in commercial zones: 89.7%
Average demand satisfied in peripheral zones: 63.0%

Number of zones covered: 379
Demand coverage rate (all zones): 73.8%
Number of trips (objective function value): 4586 trips

Avg distance to nearest open station (ALL): 1.19 km (≈14 min walking, ≈5 min by bike)
Avg distance to nearest open station (COMMERCIAL): 0.99 km (≈12 min walking, ≈4 min by bike)
Avg distance to nearest open station (PERIPHERAL): 1.24 km (≈15 min walking, ≈5 min by bike)

Demand coverage by time period and zone type (%):
        

/var/folders/qr/mt35g7rx5s78fytrjmk0s9rh0000gn/T/ipykernel_69504/515160991.py:300: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda d: d["trips_serving_zones"].sum() / d["Demand"].sum())
/var/folders/qr/mt35g7rx5s78fytrjmk0s9rh0000gn/T/ipykernel_69504/515160991.py:300: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda d: d["trips_serving_zones"].sum() / d["Demand"].sum())
/var/folders/qr/mt35g7rx

In [18]:
import folium
import numpy as np
import pandas as pd

# ---------------------------------
# 1. Prepare data: open stations + coordinates
# ---------------------------------
stations = (
    sol[sol["open"] == 1]
    .merge(
        zones_df[["zone_id", "latitude", "longitude"]],
        on="zone_id",
        how="left"
    )
    .copy()
)

# classify station type
stations["type"] = "Other"
stations.loc[stations["zone_id"].isin(zone_groups_all), "type"] = "Peripheral"
stations.loc[stations["zone_id"].isin(commercial_zone_ids), "type"] = "Commercial"

# ---------------------------------
# 2. Map centre
# ---------------------------------
center_lat = stations["latitude"].mean()
center_lon = stations["longitude"].mean()

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=12,
    tiles="CartoDB.Voyager"
)

# ---------------------------------
# 3. Style settings
# ---------------------------------
color_map = {
    "Commercial": "#e41a1c",  # red
    "Peripheral": "#377eb8",  # blue
    "Other":      "#4daf4a"   # green
}

# scale radius a bit so sizes look reasonable
min_radius = 4
max_radius = 18
d_min = stations["docks"].min()
d_max = stations["docks"].max()

def dock_to_radius(d):
    if d_max == d_min:
        return (min_radius + max_radius) / 2
    return min_radius + (d - d_min) * (max_radius - min_radius) / (d_max - d_min)

# ---------------------------------
# 4. Add station circles
# ---------------------------------
for _, row in stations.iterrows():
    radius = dock_to_radius(row["docks"])
    stype  = row["type"]
    
    popup_html = (
        f"<b>Zone ID:</b> {row['zone_id']}<br>"
        f"<b>Type:</b> {stype}<br>"
        f"<b>Docks:</b> {int(row['docks'])}"
    )
    
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=radius,
        color=color_map[stype],
        fill=True,
        fill_color=color_map[stype],
        fill_opacity=0.8,
        weight=1,
        popup=folium.Popup(popup_html, max_width=250),
    ).add_to(m)

# ---------------------------------
# 5. Add a simple legend
# ---------------------------------
legend_html = """
<div style="
    position: fixed;
    top: 20px;
    right: 300px;   /* <-- move legend left by increasing this value */
    z-index: 9999;
    background-color: white;
    padding: 12px 16px;
    border: 1px solid #ccc;
    border-radius: 6px;
    font-size: 13px;
    line-height: 1.4;
    box-shadow: 0 2px 6px rgba(0,0,0,0.2);
">
<b style="font-size:14px;">Station type</b><br>
<div style="margin-top:6px;">
    <span style="display:inline-block;width:12px;height:12px;background:#e41a1c;
                 border-radius:2px;margin-right:6px;"></span>
    Commercial Area
</div>
<div style="margin-top:4px;">
    <span style="display:inline-block;width:12px;height:12px;background:#377eb8;
                 border-radius:2px;margin-right:6px;"></span>
    Peripheral Area
</div>
<div style="margin-top:4px;">
    <span style="display:inline-block;width:12px;height:12px;background:#4daf4a;
                 border-radius:2px;margin-right:6px;"></span>
    Other Area
</div>
<div style="margin-top:10px;font-size:12px;color:#444;">
    Circle size shows number of docks.
</div>
</div>
"""



m.get_root().html.add_child(folium.Element(legend_html))


m

#m.save("advanced_station_map.html")